# Projet maths 2022 - Q1 a Q10
## Mehdi Saidi, Pierre Prudhomme, Alexandre Wilk

### Premier pricer

In [ ]:
import math
import scipy.special

def pricer1(N, rn, hn, bn, s, f):
    q = (rn - bn)/(hn-bn)
    res = 0
    for k in range(N+1):
        res += f(s*((1+hn)**k)*((1+bn)**(N-k)))*scipy.special.comb(N, k)*(q**k)*(1-q)**(N-k) # on implémente la formule trouvée question 2
    return res/((1+rn)**N)

def f(x):
    return max(x-110, 0)


print("Le résultat de la question 4 est ", pricer1(20, 0.02, 0.05, -0.05, 100, f))

Le résultat de la question 4 est  26.616941360258558


### Second Pricer

In [ ]:
import numpy as np

def pricer2(N, rn, hn, bn, s, f):
    memoireprix = [] #On créé une matrice pour se souvenir de tout les prix à chaque noeud à chaque étape (utile pour question 6 cf le pdf)
    q = (rn - bn)/(hn-bn)
    vn = np.zeros(N+1)
    for i in range(N+1):
        vn[i] = f(s*((1+hn)**(N-i))*(1+bn)**i) #On initie les valeurs de f(Stn) à la fin de l'arbre vn[0] est le prix si l'actif n'a que augmenté
    memoireprix.append(vn)                                                                          #vn[n] est le prix si l'actif n'a que diminué
    for k in range(N-1, -1, -1):
        v = np.zeros(k+1) #On construit l'array qui contient le prix de l'option pour tout les noeuds de l'arbre à l'étape K (il y a k+1 prix)
        for j in range(len(v)):
            v[j]= (q*memoireprix[-1][j] + (1-q)*memoireprix[-1][j+1])/(1+rn) #On construit Vk a partir de vk+1, sachant que Vk+1[i] corespond au prix
        memoireprix.append(v)                                                   # si le cours de l'actif risqué augmente, Vk+1[i+1] corespond au prix
    return memoireprix[-1][0]                                                  # si le cours de l'actif risqué diminue (par construction de vn).


print("le résultat de la question 6 est",pricer2(3, 0.02, 0.05, -0.05, 100, lambda x : max(x-100, 0)))

le résultat de la question 6 est 7.063436197239379


### Comparaison des 2 pricers

In [ ]:
N = np.random.randint(5, 16)
price1 = pricer1(N, 0.02, 0.05, -0.05, 100, f)
price2 = pricer2(N, 0.02, 0.05, -0.05, 100, f)
print("Le prix trouvé avec le premier pricer est", price1)
print("Le prix trouvé avec le second pricer est", price2)
print("Soit une différence entre les 2 de", abs(price1-price2))

Le prix trouvé avec le premier pricer est 14.834138253408359
Le prix trouvé avec le second pricer est 14.834138253408353
Soit une différence entre les 2 de 5.329070518200751e-15


### Couverture



In [19]:
def f1(x):
    return max((x-100),0)

def couverture(n,r,h,b,s,f):
    q = (r-b)/(h-b)
    c = 1/(1+r)
    ## couv correspond à un tableau qui contient toutes les couvertures des dates t_0,...,t_n-1
    couv = n*[0]
    for i in range(n) :
        couv[i] = n*[0]
    ## v correspond au vecteur des issues finales tout à droite de l'arbre de probavilités
    v = (n+1)*[0]
    for i in range(n+1):
        param = s*((1+h)**(n-i))*((1+b)**i)
        v[i] = f(param)
    # on calcule les premiers couples (alpha,beta) selon les formules établies à la question 8
    for i in range(n) :
        st = s*((1+h)**(n-1-i))*((1+b)**i)
        alpha = (v[i+1] - v[i])/((b-h)*st)
        beta = ((1+b)*v[i] - (1+h)*v[i+1])/((b-h)*(1+r)**n)
        couv[n-1][i] = (alpha,beta)
    ## on calcule ensuite tous les couples (alpha,beta) en "remontant" l'arbre
    # pour cela, on décrémente de 1 size qui correspond au nb de couples (alpha,beta) à l'intant t_size
    size = n-1
    while (size>0) :
        tmp = (size+1)*[0]
        for i in range(size+1):
            tmp[i] = c*(q*v[i]+(1-q)*v[i+1])
        v = tmp
        for i in range(size) :
            st = s*((1+h)**(size-1-i))*((1+b)**i)
            # on calcule les couples (alpha,beta) selon les formules établies à la question 9
            alpha = (v[i+1] - v[i])/((b-h)*st)
            beta = ((1+b)*v[i] - (1+h)*v[i+1])/((b-h)*(1+r)**size)
            couv[size-1][i] = (alpha,beta)
        size -= 1
    # on retourne le tableau des couples (c'est une matrice diagonale supérieure)
    return couv

couv = couverture(2, 0.03, 0.05, -0.05, 100, f1)
print("(alpha0, beta0) vaut", couv[0][0])
print("Si  le prix de l'actif risqué monte, (alpha1, beta1) vaut:", couv[1][0])
print("Si  le prix de l'actif risqué diminue, (alpha1, beta1) vaut:", couv[1][1])


(alpha0, beta0) vaut (0.7961165048543688, -73.42822132151944)
Si  le prix de l'actif risqué monte, (alpha1, beta1) vaut: (0.9761904761904762, -91.78527665189932)
Si  le prix de l'actif risqué diminue, (alpha1, beta1) vaut: (-0.0, -0.0)
